In [1]:
import os

os.chdir("../..")
print(os.getcwd())

C:\Users\Ali\OneDrive - Georgia Institute of Technology\25-5 Summer\CS 7643 - Deep Learning\_Project\mbari-mae


In [2]:
from scripts.pretrain_script import *

C:\Users\Ali\miniconda3\envs\new25\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Ali\miniconda3\envs\new25\Lib\site-packages\s3prl\upstream\byol_s\byol_a\common.py:20: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("sox_io")
ESPnet is not installed, cannot use espnet_hubert upstream


Train dataset: 1017 examples, 31 classes
  Clymene_Dolphin: 38
  Bottlenose_Dolphin: 15
  Spinner_Dolphin: 69
  Beluga,_White_Whale: 30
  Bearded_Seal: 22
  Minke_Whale: 10
  Humpback_Whale: 38
  Southern_Right_Whale: 15
  White-sided_Dolphin: 33
  Narwhal: 30
  White-beaked_Dolphin: 34
  Northern_Right_Whale: 32
  Frasers_Dolphin: 52
  Grampus,_Rissos_Dolphin: 40
  Harp_Seal: 28
  Atlantic_Spotted_Dolphin: 35
  Fin,_Finback_Whale: 30
  Ross_Seal: 30
  Rough-Toothed_Dolphin: 30
  Killer_Whale: 21
  Pantropical_Spotted_Dolphin: 40
  Short-Finned_Pacific_Pilot_Whale: 40
  Bowhead_Whale: 36
  False_Killer_Whale: 35
  Melon_Headed_Whale: 38
  Long-Finned_Pilot_Whale: 42
  Striped_Dolphin: 49
  Leopard_Seal: 6
  Walrus: 23
  Sperm_Whale: 45
  Common_Dolphin: 31
Validation dataset: 339 examples, 31 classes
  Clymene_Dolphin: 12
  Bottlenose_Dolphin: 5
  Spinner_Dolphin: 23
  Beluga,_White_Whale: 10
  Bearded_Seal: 7
  Minke_Whale: 4
  Humpback_Whale: 13
  Southern_Right_Whale: 5
  White-side

In [3]:
# sanity check
model = WMMDClassifier(
 num_classes=31,
 backbone="patrickvonplaten/tiny-wav2vec2-no-tokenizer", finetune=True,
 class_weights=class_weights,
 ckpt_path=""
)

print(ModelSummary(model, max_depth=1))

   | Name            | Type                | Params | Mode 
-----------------------------------------------------------------
0  | backbone        | Wav2Vec2Model       | 26.7 K | eval 
1  | classifier      | Sequential          | 49.2 K | train
2  | criterion       | CrossEntropyLoss    | 0      | train
3  | train_precision | MulticlassPrecision | 0      | train
4  | train_recall    | MulticlassRecall    | 0      | train
5  | train_f1        | MulticlassF1Score   | 0      | train
6  | val_precision   | MulticlassPrecision | 0      | train
7  | val_recall      | MulticlassRecall    | 0      | train
8  | val_f1          | MulticlassF1Score   | 0      | train
9  | test_precision  | MulticlassPrecision | 0      | train
10 | test_recall     | MulticlassRecall    | 0      | train
11 | test_f1         | MulticlassF1Score   | 0      | train
-----------------------------------------------------------------
75.9 K    Trainable params
0         Non-trainable params
75.9 K    Total params
0.304  

In [4]:
# sanity check
model = WMMDClassifier(
 num_classes=31,
 backbone_lr=1e-5,
 head_lr=1e-4,
 backbone="mae-ast",
 finetune=True,
 class_weights=class_weights,
 ckpt_path="notebook/downstream/load_model/random-4en1de.pt"
)

print(ModelSummary(model, max_depth=1))

   | Name            | Type                | Params | Mode 
-----------------------------------------------------------------
0  | backbone        | UpstreamExpert      | 36.0 M | train
1  | classifier      | Sequential          | 6.3 M  | train
2  | criterion       | CrossEntropyLoss    | 0      | train
3  | train_precision | MulticlassPrecision | 0      | train
4  | train_recall    | MulticlassRecall    | 0      | train
5  | train_f1        | MulticlassF1Score   | 0      | train
6  | val_precision   | MulticlassPrecision | 0      | train
7  | val_recall      | MulticlassRecall    | 0      | train
8  | val_f1          | MulticlassF1Score   | 0      | train
9  | test_precision  | MulticlassPrecision | 0      | train
10 | test_recall     | MulticlassRecall    | 0      | train
11 | test_f1         | MulticlassF1Score   | 0      | train
-----------------------------------------------------------------
42.4 M    Trainable params
0         Non-trainable params
42.4 M    Total params
169.438

In [5]:
# model configurations
model_configs = [
    # {"num_classes": num_classes, "lr": 1e-3, "backbone": "facebook/wav2vec2-base",  "finetune": False, "class_weights": class_weights, "max_epochs": 500, "ckpt_path": ""},
    {"num_classes": num_classes, "lr": 1e-6, "backbone": "mae-ast",  "finetune": True, "class_weights": class_weights, "max_epochs": 400, "ckpt_path": "random-4en1de.pt"},
    # {"num_classes": num_classes, "lr": 1e-3, "backbone": "patrickvonplaten/tiny-wav2vec2-no-tokenizer",  "finetune": True, "class_weights": class_weights, "max_epochs": 500, "ckpt_path": ""},
]

## 2 Encoders - Random weights (scratch)

Starting with same arch. as our pretrained 2 encoder model, just with random weights

In [9]:
# model configurations
model_configs_rand_2 = [
    {"num_classes": num_classes,
     "weight_decay":  0.05,
     "backbone_lr":   1e-4,
     "head_lr":       1e-4,
     "backbone": "mae-ast",
     "finetune": True,
     "class_weights": class_weights,
     "max_epochs": 20,
     "ckpt_path": "notebook/downstream/load_model/random-2en1de-seed42.pt"},
]
set_seed(42)
train_finetune(model_configs_rand_2, early_stopping=False)

Seed set to 42
Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

   | Name            | Type                | Params | Mode 
-----------------------------------------------------------------
0  | backbone        | UpstreamExpert      | 21.9 M | train
1  | classifier      | Sequential          | 6.3 M  | train
2  | criterion       | CrossEntropyLoss    | 0      | train
3  | train_precision | MulticlassPrecision | 0      | train
4  | train_recall    | MulticlassRecall    | 0      | train
5  | train_f1        | MulticlassF1Score   | 0      | train
6  | val_precision   | M

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.5899705290794373
         test_f1            0.5231074690818787
        test_loss           1.3964496850967407
     test_precision         0.5830875635147095
       test_recall          0.4931170344352722
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Artifacts saved to notebook\downstream\model\mae-ast_imbalance\20250708_202359/
Completed mae-ast (FT), artifacts in notebook\downstream\model\mae-ast_imbalance\20250708_202359


## 2 Encoders - Pretrained

In [ ]:
# model configurations
model_configs_2en1de = [
    {"num_classes": num_classes,
     "weight_decay":  0.05,
     "backbone_lr":   1e-5,
     "head_lr":       1e-4,
     "backbone": "mae-ast",
     "finetune": True,
     "class_weights": class_weights,
     "max_epochs": 20,
     "ckpt_path": "notebook/downstream/load_model/2en1de.pt"},
]
set_seed(42)
train_finetune(model_configs_2en1de, early_stopping=False)

## 4 Encoders - Random weights (scratch)

Starting with same arch. as our pretrained 4 encoder model, just with random weights

In [10]:
# model configurations
model_configs_rand_4 = [
    {"num_classes": num_classes,
     "weight_decay":  0.05,
     "backbone_lr":   1e-4,
     "head_lr":       1e-4,
     "backbone": "mae-ast",
     "finetune": True,
     "class_weights": class_weights,
     "max_epochs": 20,
     "ckpt_path": "notebook/downstream/load_model/random-4en1de-seed42.pt"},
]
set_seed(42)
train_finetune(model_configs_rand_4, early_stopping=False)

Seed set to 42
Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

   | Name            | Type                | Params | Mode 
-----------------------------------------------------------------
0  | backbone        | UpstreamExpert      | 36.0 M | train
1  | classifier      | Sequential          | 6.3 M  | train
2  | criterion       | CrossEntropyLoss    | 0      | train
3  | train_precision | MulticlassPrecision | 0      | train
4  | train_recall    | MulticlassRecall    | 0      | train
5  | train_f1        | MulticlassF1Score   | 0      | train
6  | val_precision   | M

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6047197580337524
         test_f1            0.5329402685165405
        test_loss           1.4297627210617065
     test_precision          0.595378577709198
       test_recall          0.5024582147598267
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Artifacts saved to notebook\downstream\model\mae-ast_imbalance\20250708_203143/
Completed mae-ast (FT), artifacts in notebook\downstream\model\mae-ast_imbalance\20250708_203143


## 4 Encoders - Pretrained


In [ ]:
# model configurations
model_configs_4en1de = [
    {"num_classes": num_classes,
     "weight_decay":  0.05,
     "backbone_lr":   1e-5,
     "head_lr":       1e-4,
     "backbone": "mae-ast",
     "finetune": True,
     "class_weights": class_weights,
     "max_epochs": 20,
     "ckpt_path": "notebook/downstream/load_model/4en1de.pt"},
]
set_seed(42)
train_finetune(model_configs_4en1de, early_stopping=False)

## 6 Encoders - Random weights (scratch)

Starting with same arch. as our pretrained 6 encoder model, just with random weights

In [11]:
# model configurations
model_configs_rand_6 = [
    {"num_classes": num_classes,
     "weight_decay":  0.05,
     "backbone_lr":   1e-4,
     "head_lr":       1e-4,
     "backbone": "mae-ast",
     "finetune": True,
     "class_weights": class_weights,
     "max_epochs": 20,
     "ckpt_path": "notebook/downstream/load_model/random-6en1de-seed42.pt"},
]
set_seed(42)
train_finetune(model_configs_rand_6, early_stopping=False)

Seed set to 42
Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

   | Name            | Type                | Params | Mode 
-----------------------------------------------------------------
0  | backbone        | UpstreamExpert      | 50.2 M | train
1  | classifier      | Sequential          | 6.3 M  | train
2  | criterion       | CrossEntropyLoss    | 0      | train
3  | train_precision | MulticlassPrecision | 0      | train
4  | train_recall    | MulticlassRecall    | 0      | train
5  | train_f1        | MulticlassF1Score   | 0      | train
6  | val_precision   | M

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.5280236005783081
         test_f1            0.4444445073604584
        test_loss            1.596319556236267
     test_precision         0.5176990628242493
       test_recall          0.4085546135902405
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Artifacts saved to notebook\downstream\model\mae-ast_imbalance\20250708_204122/
Completed mae-ast (FT), artifacts in notebook\downstream\model\mae-ast_imbalance\20250708_204122


## 6 Encoders - Pretrained

In [ ]:
# model configurations
model_configs_6en1de = [
    {"num_classes": num_classes,
     "weight_decay":  0.05,
     "backbone_lr":   1e-5,
     "head_lr":       1e-4,
     "backbone": "mae-ast",
     "finetune": True,
     "class_weights": class_weights,
     "max_epochs": 20,
     "ckpt_path": "notebook/downstream/load_model/6en1de.pt"},
]
set_seed(42)
train_finetune(model_configs_6en1de, early_stopping=False)